In [105]:
# !pip install webdriver_manager
# !pip install pandas
# !pip install selenium
# !pip install pillow

In [120]:
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import requests
import csv
import os
from PIL import Image
from io import BytesIO

In [13]:
chrome_options = Options() # 크롬 드라이버 옵션 지정 객체 생성
chrome_options.add_experimental_option("detach", True) # 브라우저 꺼짐 방지

# 가톨릭관동대학교 국제성모병원 url
url = 'https://www.ish.or.kr/main/part/list.do'
driver = webdriver.Chrome(options=chrome_options) # 크롬 드라이버 옵션 설정
driver.implicitly_wait(10) # 페이지 로딩 10초 대기
driver.get(url) # 실행

In [14]:
# 1. 과 검색
search_box = driver.find_element(By.ID, 'v_keyword')
search_box.clear() # 검색어 삭제

department = "신경과"

search_box.send_keys(department) # 검색어 입력창에 department 변수 전송

In [15]:
# 2. 검색 버튼 클릭
search_button = driver.find_element(By.CLASS_NAME, 'srch_btn')
search_button.click() # 버튼 클릭

In [26]:
# 3. 과 이름 위로 마우스 이동
mouse = driver.find_element(By.ID, "part59") # part59 ~ 1씩 증가

# 4. 의료진 버튼 클릭

# 해당 과에 대한 CSS 선택자 생성
css_selector = f'a[title="{department}"]'

button = driver.find_element(By.CSS_SELECTOR, css_selector)
act = ActionChains(driver)
act.move_to_element(mouse).click(button).perform()

time.sleep(1)

In [29]:
# 3. 과 이름 위로 마우스 이동

## 정규표현식을 사용하여 ID 패턴 정의
id_pattern = re.compile(r'part\d+')
elements = driver.find_elements(By.XPATH, '//*[@id]')

for element in elements:
    element_id = element.get_attribute('id')
    if id_pattern.match(element_id):
        mouse = element
        break

# 4. 의료진 버튼 클릭

## 해당 과에 대한 CSS 선택자 생성
css_selector = f'a[title="{department}"]'

button = driver.find_element(By.CSS_SELECTOR, css_selector)
act = ActionChains(driver)
act.move_to_element(mouse).click(button).perform()

time.sleep(1)

In [34]:
# 5. 데이터 가져오기
# 진료과 교수명 사진 직위 진료분야

# 5.1. html 파싱하기

html = driver.page_source # 페이지 소스 리턴
soup = BeautifulSoup(html, 'html.parser')
soup # 페이지 소스 출력

<html lang="ko" style="height: 100%;"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="ko" name="language"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, user-scalable=yes" name="viewport"/>
<meta content="Mon, 06 Jan 1990 00:00:01 GMT" http-equiv="Expires"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="no-cache" http-equiv="Cache-Control"/>
<meta content="d8050dcfda932915e01a1436eb56806476d74191" name="naver-site-verification"/>
<!-- Google tag (gtag.js) -->
<script src="https://partner.googleadservices.com/gampad/cookie.js?domain=www.ish.or.kr&amp;callback=_gfp_s_&amp;client=ca-pub-1834082610234299&amp;cookie=ID%3Df82f4559947ecc4c-22bd08bd8ee400fe%3AT%3D1697090647%3ART%3D1697091814%3AS%3DALNI_MYlU_lCl7HitzQhsoz8_CGUTVdcdQ&amp;gpic=UID%3D00000c5d05a19b8a%3AT%3D16970906

In [121]:
# 의료진 정보를 저장할 리스트 초기화
doctor_info_list = []

# doctor_info1부터 doctor_info200까지 반복하며 정보 추출
for i in range(1, 200):
    doctor_id = f"doctor_info{i}"
    doctor_elem = soup.find('li', {'id': doctor_id})
    
    if doctor_elem:
        # 실제 정보를 포함하고 있는 하위 요소들을 추출
        part_elem = doctor_elem.find('p', class_='part')
        name_elem = doctor_elem.find('p', class_='name')
        img_elem = doctor_elem.find('img', {'src': True})
        clinic_elem = doctor_elem.find('p', class_='clinic')
        
        # 정보가 존재하는 경우 변수에 저장
        if part_elem and name_elem and img_elem and clinic_elem:
            dptm = part_elem.text.strip()  # '신경과'
            name = name_elem.text.strip().split()[0]  # '구본대'
            image_name = img_elem['src'].split('/')[-1]  # '545828s.png'
            position = name_elem.text.strip().split()[-1]  # '교수'
            clinic = clinic_elem.text.strip()  # '퇴행성 척추질환(목, 허리 디스크), ...'
            img_src = "https://www.ish.or.kr" + img_elem['src']

            # 이미지를 다운로드
            response = requests.get(img_src)
            if response.status_code == 200:
                image_data = response.content
                image = Image.open(BytesIO(image_data))
                image.save(f"02/images/{image_name}")  # 이미지를 로컬 파일로 저장

            # 정보를 딕셔너리로 저장
            doctor_info = {
                'dptm': dptm,
                'name': name,
                'image': image_name,  # 이미지 파일 이름으로 저장
                'position': position,
                'clinic': clinic,
            }
            doctor_info_list.append(doctor_info)

# doctor_info_list에 정보가 저장됨

# CSV 파일로 내보내기
with open("02/doctor_info.csv", 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['dptm', 'name', 'image', 'position', 'clinic']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for doctor_info in doctor_info_list:
        writer.writerow(doctor_info)
